In [8]:
import nutils as ntl
import numpy as np
import matplotlib.pyplot as plt
import inspect


## SUPG

We use SUPG(iNsErT mEtHoD hErE) to solve the advection dominated skew to mesh benchmark problem as described in [links go here]:

$$-\kappa \Delta u + \beta\cdot \nabla u = 0  $$

with $\kappa = 10^{-3}$, $\beta = (.5, 1)$ on a square grid with dirichlet boundary condition 

$$u = \begin{cases} 1 & y = 0 \text{ or } x = 0 \text{ and } y < 0.2 \\ 0 & \text{otherwise} \end{cases}   $$ 

In [41]:
degree = 1 #degree of basis functions
nelems = 9 #number of elements along edge of unit square
etype = 'square'

topo, geom =  ntl.mesh.unitsquare(nelems, etype) #rectilinear([np.linspace(0,1, nx), np.linspace(0,1, ny) ])
ns = ntl.function.Namespace()
ns.x = geom
ns.beta= np.array([0.5, 1.0])
ns.kappa = 1e-3
ns.basis = topo.basis('lagrange', degree = degree)
ns.u = 'basis_i ?dofs_i'

#element intrinsic time scale
ns.h = 0.5*np.sqrt(2/nelems**2) #mesh size parameter
ns.betanorm = 'beta_i beta_i'
ns.tau = 'h / (2 betanorm)'

#dirichlet boundary condition
ns.badbnd = topo.boundary['left'].indicator(topo.boundary['left'][0:int(nelems/5)])

'''
print(topo.boundary['left'][0:int(nelems/5)])    
bezier = topo.sample('bezier', 2)
bezier2 = topo.boundary['left'].sample('bezier', 2)
x = bezier2.eval(ns.x[1])
badbnd1 = bezier2.eval(2* topo.boundary['left'].indicator(topo.boundary['left'][0:int(nelems/5)]))
print(badbnd1, x)
plt.plot(x, badbnd1, '*')
''' 

sqr = topo.boundary['bottom'].integral('(u - 1)^2 d:x' @ ns, degree = degree*2)
sqr += topo.boundary['left'].integral( '(u - badbnd)^2 d:x' @ ns, degree = degree*2 )
cons = ntl.solver.optimize('dofs', sqr, droptol=1e-15)

#residual
res = topo.integral( 'kappa basis_i,j u_,j d:x' @ ns, degree = degree * 2)
res += topo.integral( 'tau beta_k basis_i,k (-kappa u_,jj + beta_n u_,n)  d:x' @ ns, degree = degree * 2) 

dofs = ntl.solver.solve_linear('dofs', res, constrain= cons)




#'''

optimize > iter 1 > solve > solver returned with residual 5e-17
optimize > constrained 19/100 dofs, optimum value 1.60e-02
solve > solver returned with residual 7e-17


In [37]:
#print((beta *beta).sum(0)) #ct.getfullargspec(topo.basis))
#elp(np.dot)#(topo.basis)
help(ntl.solver.solve_linear)
#print(topo.boundary)

Help on function solve_linear in module nutils.solver:

solve_linear(target, residual, constrain=None, *, arguments={}, solveargs={})
    solve linear problem
    
    Parameters
    ----------
    target : :class:`str`
        Name of the target: a :class:`nutils.function.Argument` in ``residual``.
    residual : :class:`nutils.sample.Integral`
        Residual integral, depends on ``target``
    constrain : :class:`numpy.ndarray` with dtype :class:`float`
        Defines the fixed entries of the coefficient vector
    arguments : :class:`collections.abc.Mapping`
        Defines the values for :class:`nutils.function.Argument` objects in
        `residual`.  The ``target`` should not be present in ``arguments``.
        Optional.
    
    Returns
    -------
    :class:`numpy.ndarray`
        Array of ``target`` values for which ``residual == 0``

